In [ ]:
# uncomment as necessary
#!pip3 install -r requirements.txt
#!pip3 install pylabel
# !pip3 install wandb

In [ ]:
# this must be run from the command line and the wandb key inserted
# !python3 -m wandb login

In [ ]:
import json
from IPython import display
import os
from pylabel import importer
import shutil

In [ ]:
annotations_location = './Dataset/annotations/instances_val2017.json'
images_location = './Dataset/val2017'

with open(annotations_location) as json_file:
    coco_val = json.load(json_file)

In [ ]:
# display.Image(os.path.join('.',images_location,coco_val['images'][1500]['file_name']))

In [ ]:
# careful this assumes annotations json is in same directory as images due to bug in pylabel
dataset = importer.ImportCoco("Dataset/val2017/instances_val2017.json", path_to_images="", name="val2017coco")
dataset.df.head(5)

In [ ]:
#random state set at 42, the meaning of life
dataset.splitter.GroupShuffleSplit(train_pct=.9, test_pct=0.1, val_pct=0,random_state=42)

In [ ]:
dataset.export.ExportToYoloV5(output_path='Dataset_yolo/labels',yaml_file='dataset_yolo.yaml', copy_images=True, use_splits=True)

In [ ]:
# copy custom yaml file into Yolo dataset directory
shutil.copyfile('dataset_yolo.yaml', 'Dataset_yolo/dataset_yolo.yaml')

In [ ]:
# training
!python3 train.py --workers 4 --device cpu --epochs 20 --batch-size 4 --data Dataset_yolo/dataset_yolo.yaml --img 640 640 --cfg cfg/training/yolov7-custom.yaml --weights 'weights/yolov7.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

In [ ]:
# inference
!python3 detect.py --weights weights/yolov7.pt --conf 0.25 --img-size 640 --source Dataset/val2017/000000037777.jpg